In [1]:
import pandas as pd
import numpy as np
import pickle
import os

from sklearn.preprocessing import MinMaxScaler

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold

In [2]:
# data = pd.read_csv("../Datasets/ratings.csv")
# data

In [3]:
# train , test = train_test_split(data, random_state=123, test_size=0.25)
# train

In [4]:
# train.to_csv("../Datasets/train_ratings.csv", index=False)
# test.to_csv("../Datasets/test_ratings.csv", index=False)

#### Model

In [5]:
with open("../Content_based_RS/artifacts/data_boardgame_content_based.pkl", "rb") as f :
    data_cb = pickle.load(f)
with open("../Content_based_RS/artifacts/similarity_boardgame_content_based.pkl", "rb") as f :
    similarity_cb = pickle.load(f)
data_cb

,boardgame_id,boardgame_name,main_tags
0,30549,Pandemic,in pandem sever virul diseas have broken out s...
1,822,Carcassonne,carcassonn is a tileplac game in which the pla...
2,13,Catan,in catan formerli the settler of catan player ...
3,68448,7 Wonders,you are the leader of one of the 7 great citi ...
4,36218,Dominion,quotyou are a monarch like your parent befor y...
...,...,...,...
20583,16874,Vince Lombardi's Game,quotvinc lombardi gamequot is a footbal simul ...
20584,3589,Beast Wars Transformers Mutating Card Game,from the boxth beast war have begun choos side...
20585,296892,Sacred Rites,it ha come time to honor the tradit of the pas...
20586,10052,Bongo Kongo,each player take a posit adjac to one of the f...


In [6]:
with open("../Collaborative_Filtering_RS/artifacts/model_collaborative_filtering_based_RS.pkl", "rb") as f :
    model_cfb = pickle.load(f)
with open("../Collaborative_Filtering_RS/artifacts/bg_pivot_collaborative_filtering_based_RS.pkl", "rb") as f :
    bg_pivot_cfb = pickle.load(f)
bg_pivot_cfb

,1 Family Meeple,1000rpm,100pcBlade,1point21gigawatts,2ndPlace,3dicebombers,4Corners,521tiger,5th Beatles,ALGO,...,zaubererer,zautman,zb1035,zedeemia,zefquaavius,zeotter,zgabor,zombiegod,zumba,zumikon
boardgame_name,,,,,,,,,,,,,,,,,,,,,
10 Days in Africa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Days in Asia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Days in Europe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Days in the USA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11 nimmt!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zombie in My Pocket,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0
Zombies!!!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
Zooloretto,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
def collaborative_filtering_RS(model, boardgame_name, bg_pivot, k):
    bg_name_list = []
    bg_id_list = []
    distance_list = []
    boardgame_idx = np.where(bg_pivot.index == boardgame_name)[0][0]
    distances, boardgame_idx_corr = model.kneighbors(bg_pivot.iloc[boardgame_idx, :].values.reshape(1,-1), n_neighbors=k)
    for i in range(len(boardgame_idx_corr)):
        boardgames = bg_pivot.index[boardgame_idx_corr[i]]
        distances_1 = distances[i]
        zipped = list(zip(boardgames[1:], distances_1[1:]))
        sort = sorted(zipped, key=lambda x : x[1])
        for bg_name, distance in sort:
            bg_name_list.append(bg_name)
            distance_list.append(distance)
    return bg_name_list, distance_list


In [8]:
def recommendation_system_cb(data, similary, boardgame, k):
    bg_name_list = []
    bg_id_list = []
    distance_list = []
    index  = data[data["boardgame_name"] == boardgame].index[0]
    distances = sorted(list(enumerate(similary[index])), reverse= True, key= lambda x : x[1])
    for distance in distances[1:k]:
        # print(data.iloc[distance[0]]["boardgame_name"])
        bg_id_list.append(data.iloc[distance[0]]["boardgame_id"])
        bg_name_list.append(data.iloc[distance[0]]["boardgame_name"])
        distance_list.append(distance[1])
    return bg_id_list, bg_name_list, distance_list

In [9]:
data_weight = pd.read_csv("../Datasets/weight_hybdrid_RS_dataset.csv")
data_weight = data_weight[['boardgame_id', 'boardgame_name', 'image', 'weight_ratings']]
data_weight

,boardgame_id,boardgame_name,image,weight_ratings
0,115746,War of the Ring: Second Edition,https://cf.geekdo-images.com/ImPgGag98W6gpV1KV...,8.670156
1,224517,Brass: Birmingham,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,8.620723
2,220308,Gaia Project,https://cf.geekdo-images.com/hGWFm3hbMlCDsfCsa...,8.537215
3,246900,Eclipse: Second Dawn for the Galaxy,https://cf.geekdo-images.com/Oh3kHw6lweg6ru71Q...,8.476829
4,174430,Gloomhaven,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,8.470849
...,...,...,...,...
20771,276022,Alien: USCSS Nostromo,https://cf.geekdo-images.com/b7cH4UQM7PFraumDd...,3.367860
20772,11901,Tic-Tac-Toe,https://cf.geekdo-images.com/UImMYmMZKE4AGTMPH...,3.228797
20773,5432,Chutes and Ladders,https://cf.geekdo-images.com/P1qJDS_DFTtP_FrpW...,3.200951
20774,7316,Bingo,https://cf.geekdo-images.com/7xqN6StcQz1FoGplo...,3.079465


In [10]:
def hybrid_rs(data_cb, data_weight, similarity_cb,model_cfb,  boardgame,bg_pivot_cfb, k):
    boardgame_id_cb,boardgame_name_cb, similarity_k_cb = recommendation_system_cb(data_cb, similary=similarity_cb, boardgame=boardgame, k=k)
    similarity_dict_cb = {"boardgame_id": boardgame_id_cb,"boardgame_name" :boardgame_name_cb, "similarity_cb" : similarity_k_cb}
    similarity_df_cb = pd.DataFrame(similarity_dict_cb)

    boardgame_name_cfb, similarity_k_cfb = collaborative_filtering_RS(model_cfb, boardgame, bg_pivot_cfb, k)
    similarity_dict_cfb = {"boardgame_name" :boardgame_name_cfb, "similarity_cfb" : similarity_k_cfb}
    similarity_df_cfb = pd.DataFrame(similarity_dict_cfb)

    similarity_hb = similarity_df_cb.merge(similarity_df_cfb, on="boardgame_name")

    data_rs = similarity_hb.merge(data_weight , on = ["boardgame_id", "boardgame_name"])
    Scaling = MinMaxScaler()
    data_scaled_rs = Scaling.fit_transform(data_rs[["similarity_cb", "similarity_cfb", "weight_ratings"]])
    data_normalized_rs = pd.DataFrame(data_scaled_rs, columns=["similarity_cb", "similarity_cfb", "weight_ratings"])
    data_rs[["similarity_normalized_cb", "similarity_normalized_cfb", "weight_normalized_ratings"]] = data_normalized_rs
    data_rs["score_total"] = data_rs["similarity_normalized_cb"] * 1/3 + data_rs["similarity_normalized_cfb"] * 1/3 + data_rs["weight_normalized_ratings"] * 1/3
    data_rs = data_rs.sort_values(by = "score_total", ascending= False)
    data_rs_final = data_rs[['boardgame_id', 'boardgame_name', 'image','score_total']]
    return data_rs_final

In [11]:
def corr_pearson(bg_pivot_test):
    # Chuyển DataFrame thành mảng NumPy
    matrix = bg_pivot_test.to_numpy()


    # Tính ma trận tương quan Spearman
    correlation_matrix = bg_pivot_test.T.corr(method="spearman")
    print(correlation_matrix)
    # Tạo DataFrame từ ma trận tương quan với tên cột
    correlation_df = pd.DataFrame(correlation_matrix, columns=bg_pivot_test.index, index=bg_pivot_test.index)

    print("Ma trận tương quan:")
    display(correlation_df)
    
    return correlation_df

In [12]:
def item_relevant(correlation_df):
    bg_corr = {}
    bg_corr_non_rate = {}
    for i in correlation_df.columns:
        value = correlation_df[correlation_df[i] > 0.1][i].values
        boardgame_corr = correlation_df[correlation_df[i] > 0.1][i].index
        boardgame_corr = [i for i in boardgame_corr]
        zipped = list(zip(boardgame_corr[1:], value[1:]))
        sort = sorted(zipped, key=lambda x : x[1], reverse=True)
        bg_name = [i for i, j in sort]
        bg_corr[i] = sort
        bg_corr_non_rate[i] = bg_name
    
    return bg_corr, bg_corr_non_rate


In [13]:
def precision_at_k(actual, predicted, k):
    # Take top-k predicted items
    predicted_k = predicted[:k]
    # Count the number of common items between actual and predicted
    common_items = set(actual) & set(predicted_k)
    # Calculate precision@k
    precision = len(common_items) / k if k > 0 else 0
    return precision

def recall_at_k(actual, predicted, k):
    # Take top-k predicted items
    predicted_k = predicted[:k]
    # Count the number of common items between actual and predicted
    common_items = set(actual) & set(predicted_k)
    # Calculate recall@k
    recall = len(common_items) / len(actual) if len(actual) > 0 else 0
    return recall

In [14]:
def collaborative_filtering_RS(model, boardgame_name, bg_pivot, k):
    bg_names = []
    boardgame_idx = np.where(bg_pivot.index == boardgame_name)[0][0]
    # print(boardgame_idx)
    distance, boardgame_idx_corr = model.kneighbors(bg_pivot.iloc[boardgame_idx, :].values.reshape(1,-1), n_neighbors=k + 1)
    ## Item-based Recommendation Systems
    col_names = bg_pivot.index[boardgame_idx_corr[0]].tolist()
    zipped = list(zip(col_names[1:], distance[0][1:])) # bỏ chính nó
    sort = sorted(zipped, key=lambda x : x[1], reverse=True)
    # print(sort)
    # return
    # items_dict[boardgame_name] = sort
    for i in range(len(sort)):
        bg_names.append(sort[i][0])
    return bg_names
# l = collaborative_filtering_RS(model,  "10 Days in Africa", bg_pivot, k)
# l

In [15]:
test = pd.read_csv("../Datasets/test_ratings.csv")
test

,user_name,rating,comment,boardgame_id,boardgame_name
0,ryanabrams,8.0,This is a ton of brilliant stuff pasted onto a...,222862,Archmage
1,anfdeng,7.5,狼人的替代品,41114,The Resistance
2,ogrim,8.0,Very close to the Mystery Rummy series with so...,878,Wyatt Earp
3,dscford,8.5,In many ways this game has the burden of livin...,177736,A Feast for Odin
4,Innsmouth_Look,6.0,French version of Fast Flowing Forest Fellers ...,38548,Fast Flowing Forest Fellers
...,...,...,...,...,...
168426,bretcliftawn,5.0,Yup. Draw cards and wait till others get toget...,9209,Ticket to Ride
168427,s_nav,4.0,First impressions : 4 / 10. Not so good Some u...,144041,Patchistory
168428,Tomay,7.0,"Playable game of the Iran-Iraq War, but tends ...",10102,"Ignorant Armies: The Iran-Iraq War, 1980-88"
168429,SteveK2,7.0,Great little game where you’re always torn bet...,32412,Palastgeflüster


In [16]:
test = test[['user_name', 'rating',  'boardgame_name']]
## Create Matrix for the User_Boardgame Interaction using pivot function
bg_pivot_test = test.pivot_table(columns="user_name", index="boardgame_name", values="rating")
bg_pivot_test.fillna(0.0, inplace= True)

In [ ]:
correlation_df_test = corr_pearson(bg_pivot_test)
_ , dictionary_item_relevants = item_relevant(correlation_df_test)
dictionary_item_relevants

In [ ]:
ks = [5, 10] # Value of k for precision@k and recall@k
precision_list_at_k = []
recall_list_at_k = []
for k in ks:
    predictions = {}
    actuals = {}
    for i in bg_pivot_test.index:
        list_pred = hybrid_rs(data_cb, data_weight, similarity_cb,model_cfb,  i,bg_pivot_cfb, k)
        predictions[i] = list_pred
        actuals[i] = dictionary_item_relevants[i]
    
    precision_list = []
    recall_list = []
    for i, y in actuals.items():
        precision_k = precision_at_k(actuals[i], predictions[i], k)
        recall_k = recall_at_k(actuals[i], predictions[i], k)
        precision_list.append(precision_k)
        recall_list.append(recall_k)



    precision = sum(precision_list) / len(precision_list)
    recall = sum(recall_list) / len(recall_list)
    precision_list_at_k.append(precision)
    recall_list_at_k.append(recall)
    print(f"Precision@{k}: {precision}")
    print(f"Recall@{k}: {recall}")
    print(len(precision_list))
    print(recall_list)
# actuals["10 Days in Africa"]

In [ ]:
precision_list = []
recall_list = []
for i, y in actuals.items():
    precision_k = precision_at_k(actuals[i], predictions[i], k)
    recall_k = recall_at_k(actuals[i], predictions[i], k)
    precision_list.append(precision_k)
    recall_list.append(recall_k)



precision = sum(precision_list) / len(precision_list)
recall = sum(recall_list) / len(recall_list)
print(f"Precision@{k}: {precision}")
print(f"Recall@{k}: {recall}")
print(len(precision_list))
print(recall_list)
actuals["10 Days in Africa"]